# Unit 2

## Diff Parser: Breaking Down Code Changes for Review

## Processing Diff Lines for Code Review

## Extracting File Paths from Diff Headers

## Parsing Hunk Headers for Diff Analysis

## Parsing Complete Hunks for Code Review

## Handling Multiple Hunks in Diff Parsing

## Handling Multiple Hunks in Diff Parsing